### Tensorflow基礎用法

In [2]:
import tensorflow as tf

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 建立tensorflow(簡稱tf)字串常數
c = tf.constant('Hello, world!')

In [4]:
# 直接印變數只會顯示tf的變數類型
print(c)

Tensor("Const:0", shape=(), dtype=string)


In [5]:
# 執行tf變數前,需建立tf session(一個session可以部署到多台機器上執行)
# with語法可在獨立的命名空間,定義變數(名為sess)的存活範圍
# run()可以啟動事先定義好的tf動作(operations),許多動作構成計算圖(graph)
with tf.Session() as sess:
    print(sess.run(c))

b'Hello, world!'


---
tensor:即任意維度的向量,包括：
* scalar: 0維陣列
* vector: 向量, 1維陣列
* matrix: 矩陣, 2維陣列

---
TF計算圖中,tensor代表線段,operations代表點,構成graph
TF僅會計算需要用到的節點,節省資源加快效率

In [44]:
## 計算圖內的節點包括constants(值不可變)和variables(值可變)
x = tf.constant(5.2)
y = tf.Variable([1])
# Variable若沒有先初始化, 則一定要先assign才能使用
y = y.assign([0])
with tf.Session() as sess:
#     此處不做初始化
#     init = tf.global_variables_initializer()
#     sess.run(init)
    print('x:',x.eval())
    print('y:',y.eval())
#   用run也可以
    print('Run x:',sess.run(x),', Run y:',sess.run(y))

x: 5.2
y: [0]
Run x: 5.2 , Run y: [0]


---
可為不同目的, 自訂不同的計算圖

In [7]:
g=tf.Graph()
# 使用g作為接下來預設的計算圖
with g.as_default():
    x=tf.constant(8, name='x_name')
    y=tf.constant(9, name='y_name')
    thisAdd=tf.add(x,y, name='thisIsAddName')
    
    with tf.Session() as sess:
#       要注意不是直接用name呼叫
        print('thisAdd:',thisAdd.eval())
    

thisAdd: 17


---
矩陣運算練習

In [10]:
# 加法
with g.as_default():
#     建立一個vector
    primes=tf.constant([1,2,3,4,5,6])
#     建立一個長度為6,每格值為1之vector
    ones=tf.ones([6], dtype=tf.int32)
    
    add=tf.add(primes,ones)
    with tf.Session() as sess:
        print('Add:',add.eval())

Add: [2 3 4 5 6 7]


In [14]:
# 檢視tensor維度(shape)
with g.as_default():
#   建立純量
    scalar=tf.zeros([])
#   建立向量
    vector=tf.zeros([3])
#   建立矩陣
    matrix=tf.zeros([2,3])
    
    with tf.Session() as sess:
        print('Saclar shape:',scalar.get_shape(),',Value:',scalar.eval())
        print('Vecotr shape:',vector.get_shape(),',Value:',vector.eval())
        print('Matrix shape:',matrix.get_shape(),',Value:',matrix.eval())    

Saclar shape: () ,Value: 0.0
Vecotr shape: (3,) ,Value: [0. 0. 0.]
Matrix shape: (2, 3) ,Value: [[0. 0. 0.]
 [0. 0. 0.]]


---
Brodcasting:將運算套用到整個tensor上

In [17]:
with g.as_default():
    primes=tf.constant([1,2,3,4,5,6], dtype=tf.int32)
#   此處只有單一值
    one=tf.constant(1, dtype=tf.int32)
#   加法運算會被套用到整個scalar上
    add=tf.add(one,primes)
    with tf.Session() as sess:
        print('Result:',add.eval())

Result: [2 3 4 5 6 7]


---
矩陣乘法

In [20]:
with g.as_default():
#   要注意前者的行數要等於後者的列數才能算(ex: 3x4, 4x2 相乘後變成3x2)
    x=tf.constant([[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=tf.int32)
    y=tf.constant([[1,1],[2,2],[3,3],[4,4]], dtype=tf.int32)
    
    mutiply=tf.matmul(x,y)
    with tf.Session() as sess:
        print('x:',x.eval())
        print('y:',y.eval())
#       第一列就是1x1+2x2+3x3+4x4=30,第二列就是5x1+6x2+7x3+8x4=70,etc...
        print('Mutiply Result:',mutiply.eval())

x: [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
y: [[1 1]
 [2 2]
 [3 3]
 [4 4]]
Mutiply Result: [[ 30  30]
 [ 70  70]
 [110 110]]


---
Reshape:重新排列tensor的維度

In [31]:
with g.as_default():
    number=tf.constant([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], dtype=tf.int32)
    num2x8=tf.reshape(number,[2,8])
    num2x2x4=tf.reshape(number,[2,2,4])
    num16=tf.reshape(num2x2x4,[16])
    
    with tf.Session() as sess:
        print('Number shape:',number.get_shape(),'Value:',number.eval())
        print('Num2x8 shape:',num2x8.get_shape(),'\nValue:',num2x8.eval())
        print('Num2x2x4 shape:',num2x2x4.get_shape(),'\nValue:',num2x2x4.eval())
        print('Num16 shape:',num16.get_shape(),'\nValue:',num16.eval())

Number shape: (16,) Value: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Num2x8 shape: (2, 8) 
Value: [[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]]
Num2x2x4 shape: (2, 2, 4) 
Value: [[[ 1  2  3  4]
  [ 5  6  7  8]]

 [[ 9 10 11 12]
  [13 14 15 16]]]
Num16 shape: (16,) 
Value: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


---
變數(Variable)的使用

In [47]:
with g.as_default():
#   建立一個預設值為3的變數
    x=tf.Variable([3])
#   建立一個變數,預設值為指定常態分配下的隨機亂數
    y=tf.Variable(tf.random_normal([1],mean=1.0,stddev=0.35))
    
    with tf.Session() as sess:
#       Variable若沒有初始化就使用會出錯
        try:
            print('x:',x.eval())
        except:
            print("Error")
        
        print('Initialing...')
#       初始化時會一併指定亂數值(在同一session內值都會固定)
        ini=tf.global_variables_initializer()
        sess.run(ini)
        print('x:',x.eval())
        print('y:',y.eval())
        
#       再初始化時會重新指定亂數值
        print('Initialing again..')
        sess.run(ini)
        print('x:',x.eval())
        print('y:',y.eval())

Error
Initialing...
x: [3]
y: [0.87215114]
Initialing again..
x: [3]
y: [1.2630528]


也可以用assign來動態改變變數的值

In [53]:
with g.as_default():
    x=tf.Variable(3)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print('x:',x.eval())
        print('Assign new value...')
#       要用session run才會生效
        sess.run(tf.assign(x,6))
        print('x:',x.eval())

x: 3
Assign new value...
x: 6


### Refence
* [TensorFlow Hello World](https://colab.research.google.com/notebooks/mlcc/hello_world.ipynb?hl=en)
* [TensorFlow Programming Concepts](https://colab.research.google.com/notebooks/mlcc/tensorflow_programming_concepts.ipynb?hl=en)
* [Creating and Manipulating Tensors](https://colab.research.google.com/notebooks/mlcc/creating_and_manipulating_tensors.ipynb?hl=en)
